In [1]:
import torch
import random
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import argparse,os,time
import os
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.io import wavfile
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import seaborn as sns

In [2]:
data = pd.read_csv('test.csv', index_col=0)

In [3]:
data

,letter,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
id,,,,,,,,,,,,,,,,,,,,,
2049,L,0,4,0,2,4,2,3,1,0,...,2,0,4,2,2,4,3,4,1,4
2050,C,4,1,4,0,1,1,0,2,2,...,0,3,2,4,2,4,2,2,1,2
2051,S,0,4,0,1,3,2,3,0,2,...,1,3,2,0,3,2,3,0,1,4
2052,K,2,1,3,3,3,4,3,0,0,...,3,0,3,2,4,1,0,4,4,4
2053,W,1,0,1,1,2,2,1,4,1,...,4,3,1,4,0,2,1,2,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22524,P,1,2,1,1,0,0,0,2,1,...,0,1,3,0,3,0,4,3,1,4
22525,S,4,1,1,4,0,0,1,1,4,...,1,3,1,0,0,1,3,1,2,0
22526,B,4,2,1,3,2,1,3,0,4,...,3,2,3,4,1,0,3,3,1,1


In [4]:
x_data=data.loc[:,"0":"783"].values

In [5]:
x_data=x_data.astype(np.float32)

In [6]:
x_data_test=x_data

In [7]:
x_data_test=x_data_test/x_data_test.max()

In [8]:
x_data=x_data_test.astype(np.float32)

In [9]:
batch_size=1

In [10]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer_1 = nn.Conv2d(in_channels=1,out_channels=64,kernel_size=3,
                                 stride=1,padding=2)
        self.act_1 = nn.ReLU()
        self.conv2_bn1 = nn.BatchNorm2d(64)
        self.drop_1 = nn.Dropout(p=0.2)
        
        self.layer_2 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,
                                 stride=1,padding=2)
        self.act_2 = nn.ReLU()
        self.conv2_bn2 = nn.BatchNorm2d(64)
        self.drop_2 = nn.Dropout(p=0.2)
        
        self.layer_3 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,
                                 stride=1,padding=2)
        self.act_3 = nn.ReLU()
        self.conv2_bn3 = nn.BatchNorm2d(64)
        self.drop_3 = nn.Dropout(p=0.2)
        
        self.max_1=nn.MaxPool2d(2,2)
        
        self.layer_4 = nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,
                                 stride=1,padding=2)
        self.act_4 = nn.ReLU()
        self.conv2_bn4 = nn.BatchNorm2d(128)
        self.drop_4 = nn.Dropout(p=0.2)

        self.layer_5 = nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,
                                 stride=1,padding=2)
        self.act_5 = nn.ReLU()
        self.conv2_bn5 = nn.BatchNorm2d(128)
        self.drop_5 = nn.Dropout(p=0.2)
        
        self.layer_6 = nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,
                                 stride=1,padding=2)
        self.act_6 = nn.ReLU()
        self.conv2_bn6 = nn.BatchNorm2d(128)
        self.drop_6 = nn.Dropout(p=0.2)
        
        self.max_2=nn.MaxPool2d(2,2)
        
        self.layer_7 = nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,
                                 stride=1,padding=2)
        self.act_7 = nn.ReLU()
        self.conv2_bn7 = nn.BatchNorm2d(256)
        self.drop_7 = nn.Dropout(p=0.2)
        
        self.layer_8 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,
                                 stride=1,padding=2)
        self.act_8 = nn.ReLU()
        self.conv2_bn8 = nn.BatchNorm2d(256)
        self.drop_8 = nn.Dropout(p=0.2)
        
        self.layer_9 = nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,
                                 stride=1,padding=2)
        self.act_9 = nn.ReLU()
        self.conv2_bn9 = nn.BatchNorm2d(256)
        self.drop_9 = nn.Dropout(p=0.2)
        
        self.max_3=nn.MaxPool2d(2,2)
        
        self.layer_10 = nn.Conv2d(in_channels=256,out_channels=512,kernel_size=3,
                                 stride=1,padding=2)
        self.act_10 = nn.ReLU()
        self.conv2_bn10 = nn.BatchNorm2d(512)
        self.drop_10 = nn.Dropout(p=0.2)
        
        self.layer_11 = nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,
                                 stride=1,padding=2)
        self.act_11 = nn.ReLU()
        self.conv2_bn11 = nn.BatchNorm2d(512)
        self.drop_11 = nn.Dropout(p=0.2)
        
        self.layer_12 = nn.Conv2d(in_channels=512,out_channels=512,kernel_size=3,
                                 stride=1,padding=2)
        self.act_12 = nn.ReLU()
        self.conv2_bn12 = nn.BatchNorm2d(512)
        self.drop_12 = nn.Dropout(p=0.2)
        
        self.max_4=nn.MaxPool2d(2,2)
        
        self.layer_13 = nn.Conv2d(in_channels=512,out_channels=1024,kernel_size=3,
                                 stride=1,padding=2)
        self.act_13 = nn.ReLU()
        self.conv2_bn13 = nn.BatchNorm2d(1024)
        self.drop_13 = nn.Dropout(p=0.2)
        
        self.layer_14 = nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,
                                 stride=1,padding=2)
        self.act_14 = nn.ReLU()
        self.conv2_bn14 = nn.BatchNorm2d(1024)
        self.drop_15 = nn.Dropout(p=0.2)
        
        self.max_5=nn.MaxPool2d(2,2)
        
        self.layer_15 = nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=1,
                                 stride=1)
        self.act_15 = nn.ReLU()
        self.conv2_bn15 = nn.BatchNorm2d(1024)
        self.drop_16 = nn.Dropout(p=0.2)
        
        self.layer_16 = nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=3,
                                 stride=1,padding=2)
        self.act_16 = nn.ReLU()
        self.conv2_bn16 = nn.BatchNorm2d(1024)
        self.drop_16 = nn.Dropout(p=0.2)
        
        self.layer_17 = nn.Conv2d(in_channels=1024,out_channels=1024,kernel_size=1,
                                 stride=1)
        self.act_17 = nn.ReLU()
        self.conv2_bn17 = nn.BatchNorm2d(1024)
        self.drop_17 = nn.Dropout(p=0.2)
        
        

        
        self.fc_layer_1 = nn.Linear(49*1024,1000)
        self.act_18 = nn.ReLU()
        
        self.bnm1=nn.BatchNorm1d(1000)
        self.drop_18 = nn.Dropout(p=0.2)
        
        self.fc_layer_2 = nn.Linear(1000,1000)
        self.act_19 = nn.ReLU()
        
        self.bnm2=nn.BatchNorm1d(1000)
        self.drop_19 = nn.Dropout(p=0.2)
        
        self.fc_layer_3 = nn.Linear(1000,100)
        self.act_20 = nn.ReLU()
        
        self.bnm3=nn.BatchNorm1d(100)
        self.drop_20 = nn.Dropout(p=0.2)
        
        self.fc_layer_4 = nn.Linear(100,10)
        self.act_21 = nn.ReLU()

        
        
        

    def forward(self, x):
        x = x.view(batch_size,1,28,28)
        out = self.layer_1(x)
        out = self.act_1(out)
        for module in list(self.modules())[2:-14]:
            out = module(out)
        out = out.view(batch_size,-1)
        for module in list(self.modules())[-14:]:
            out = module(out)
        return out

In [11]:
model=nn.DataParallel(Model().cuda())
model = torch.load("model_ver-1.07_deep_drop")
model.eval()

Model(
  (layer_1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (act_1): ReLU()
  (conv2_bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop_1): Dropout(p=0.2, inplace=False)
  (layer_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (act_2): ReLU()
  (conv2_bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop_2): Dropout(p=0.2, inplace=False)
  (layer_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (act_3): ReLU()
  (conv2_bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop_3): Dropout(p=0.2, inplace=False)
  (max_1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer_4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (act_4): ReLU()
  (conv2_bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop_4): 

In [12]:
save_output=np.zeros(20480)
for i in range (20480):
    output=model(torch.tensor(x_data[i]).cuda()).cpu().detach().numpy()
    save_output[i]=output.argmax()

In [13]:
save_output

array([6., 9., 8., ..., 6., 2., 0.])

In [14]:
save_output=save_output.astype(int)

In [15]:
test_id=np.arange(2049,2049+20480)

In [16]:
res=np.hstack((test_id.reshape(20480,1),
               save_output.reshape(20480,1)))

In [17]:
from pandas import Series, DataFrame

In [18]:
data_save=DataFrame(res, columns=['id','digit'])

In [19]:
data_save.to_csv("submit_data_1.07_3.csv",index = False)